In [ ]:
# useful libraries
import pandas as pd
import numpy as np

from sklearn import model_selection
import keras

from PIL import Image 


In [ ]:
# Dataset
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
print(df.shape)
df.head()

In [ ]:
# split into train and valid 
train, validation = model_selection.train_test_split(
    df,
    test_size=0.3, 
    random_state=42,
    stratify=df.label.values
)

In [ ]:
# reset index on both dataframes
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)

In [ ]:
# image ids and label dictionaries
partition = dict()

partition['train'] = list(train.image_id.values)
partition['validation'] = list(validation.image_id.values)

labels = dict(zip(list(df.image_id), list(df.label)))

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(600,800), n_channels=3,
                 n_classes=5, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.asarray(Image.open('../input/cassava-leaf-disease-classification/train_images/'+ID))

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
# Parameters
params = {'dim': (600,800),
          'batch_size': 64,
          'n_classes': 5,
          'n_channels': 3,
          'shuffle': True}

In [ ]:
# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

**---------------------- Model Design > Compile > Train --------------------------------**